In [1]:
import pandas as pd
import json
import random
import matplotlib.pyplot as plt
import gc
import copy
from tqdm import tqdm
import os

In [3]:
df = pd.read_csv('amazon_reviews.csv')
df

,userName,verified,itemName,description,image,brand,feature,category,price,rating,reviewTime,summary,reviewText,vote
0,Amazon Customer,True,"Toblerone Swiss Milk Chocolate Bar, Crunchy Sa...","[""Made from deliciously decadent ingredients, ...",['https://images-na.ssl-images-amazon.com/imag...,Toblerone,[],Prime_Pantry,$1.63,5.0,2018-01-01,Five Stars,super smooth and yummy with crunchy bits,0
1,Carla,True,"Prince of Peace Organic Tea, Oolong, 100 Tea Bags","['Prince of Peace Enterprises, Inc., founded i...",['https://images-na.ssl-images-amazon.com/imag...,Prince Of Peace,[],Grocery_and_Gourmet_Food,$6.40,5.0,2018-01-01,Five Stars,Perfect for kombucha,0
2,Jaclyn,True,"Pet Champion Adjustable No-Pull Harness, Colla...","[""The Pet Champion Large/ Extra Large 22-36 in...",['https://images-na.ssl-images-amazon.com/imag...,Pet Champion,"['Features Bright Pink Lemonade pattern', 'Dua...",Pet_Supplies,$7.99,5.0,2018-01-01,she has an odd shape chest and her pull over h...,Finally a harness that fits my puppy. I really...,0
3,Christinne M,False,Koh-I-Noor Progresso Woodless Colored 24-Penci...,['Koh-I-Noor Progresso Woodless Colored 24-Pen...,['https://images-na.ssl-images-amazon.com/imag...,KOH-I-NOOR,"['Rich, pigmented, solid color encased in lacq...",Arts_Crafts_and_Sewing,$14.18,5.0,2018-01-01,Loving!!,I LOVE THEM!! I bought them at Micheals our of...,0
4,Sandy,True,"Parker Urban Premium Ebony Metal Chiseled, Bal...","[""Stunning craftsmanship based on a classic st...",['https://images-na.ssl-images-amazon.com/imag...,Parker,['Ebony black with metallic highlights and a g...,Office_Products,NaN,5.0,2018-01-01,I love this pen,"I love this pen! I love the shape of it, the f...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551154,Greg,True,Pandahall 10m(32.80 feet) Platinum Color Iron ...,"['Iron Twist Chains, Platinum Color<P>Size: Ri...",['https://images-na.ssl-images-amazon.com/imag...,PH PandaHall,"['Iron Twist Chains, Platinum Color, Ring: abo...",Arts_Crafts_and_Sewing,$8.29,2.0,2018-10-03,Two Stars,Cheap price but weak chain.,0
551155,Gayle LaRue,True,Keter Westwood Plastic Deck Storage Container ...,"['Plenty of storage and comfortable seating, a...",['https://images-na.ssl-images-amazon.com/imag...,Keter,['Outside Dimensions: 61 in. L x 28.5 in. W x ...,Patio_Lawn_and_Garden,$121.76,4.0,2018-10-03,Deck box good investment,Great deck box,0
551156,Alfred D. Wallace,True,AcuRite 06052M Remote Battery Pack for 5-in-1 ...,"[""The AcuRite remote battery pack plugs into t...",['https://images-na.ssl-images-amazon.com/imag...,AcuRite,['Makes battery replacement easy for AcuRite 5...,Patio_Lawn_and_Garden,$19.95,5.0,2018-10-03,This will keep the old man off the roof!,Putting these in my weather station. I anticip...,0
551157,Dave Dunn Wilmer,True,Keter Unity XL Indoor Outdoor Entertainment BB...,['The extra-large outdoor entertainment statio...,['https://images-na.ssl-images-amazon.com/imag...,Keter,['Dimensions: 53.1 in. L x 20.4 in. W x 35.4 i...,Patio_Lawn_and_Garden,$155.64,5.0,2018-10-03,EASY TO ASSEMBLE AND WORKS WELL NEXT TO GRILL,This is a nice quality grilling table where yo...,0


In [34]:
df_processed

,overall,reviewText,summary
0,5,Hope they come back with this scent,
1,5,"I have had this shower gel once before, and it...",Ah-Mazing!
2,3,"I like most of the soaps in this line, but thi...",Too heavily scented for me
3,5,I have used this off and on for several years ...,Curly Hair
4,3,Jim and I are over 50. I ordered the Crest Pro...,Sounded good until I read the label
...,...,...,...
1359,5,"my wife loved it ,and thanked me ,she gets goo...",
1360,5,This is one of my favorite bath and body produ...,Bath and Body Wash Gels
1361,5,I have used this product for years. It is the ...,I have used this product for years. It is ...
1362,5,"my wife loves this bath soap, too bad its too ...",Would buy more if it wasn't so hard to come by


In [52]:
import pandas as pd
import copy
from tqdm import tqdm
import os

for json_file_path in tqdm(os.listdir("."), desc="Processing files"):
    if json_file_path.endswith(".json"):
        print(json_file_path, "*************************")

        # Read the JSON file and remove duplicates
        df = pd.read_json(json_file_path, lines=True)
        df = df.drop_duplicates("reviewText")

        # Deep copy the DataFrame
        df_processed = copy.deepcopy(df)
        df_processed = df_processed[~df_processed['summary'].isna()]
        df_processed = df_processed[~df_processed['reviewText'].isna()]

        # List of strings to replace
        tmp = ["One Stars", "Two Stars", "Three Stars", "Four Stars", "Five Stars",
               "1 Stars", "2 Stars", "3 Stars", "4 Stars", "5 Stars",
               "1 stars", "2 stars", "3 stars", "4 stars", "5 stars",
               "One stars", "Two stars", "Three stars", "Four stars", "Five stars",
               "one stars", "two stars", "three stars", "four stars", "five stars"]

        # Iterate over summary and reviewText columns
        def replace_unseen(x):
            for word in tmp:
                x = x.replace(word, "")
            return x

        for column in ["summary", "reviewText"]:
            df_processed[column] = df_processed[column].apply(lambda x: replace_unseen(x))

        def generate_prompt(row):
            if row['summary'] != "":
                prompt = f"Instructions: Evaluate the following product review on a scale of 1 to 5, " \
                         f"with 1 being very negative and 5 being very positive. \nInput: a review text for a(n) " \
                         f"product\n{row['summary']}\n{row['reviewText']}\nAnswer: "
            else:
                prompt = f"Instructions: Evaluate the following product review on a scale of 1 to 5, " \
                         f"with 1 being very negative and 5 being very positive. \nInput: a review text for a(n) " \
                         f"product\n{row['reviewText']}\nAnswer: "
            return prompt

        final_df = copy.deepcopy(df_processed)
        final_df['prompt'] = final_df.apply(lambda row: generate_prompt(row), axis=1)
        final_df.rename(columns={"overall": "output_field"}, inplace=True)

        def sample_by_kind(df):
            # Assuming final_df is your DataFrame and score column contains values between 1 and 5
            # Assuming you want each category to be sampled at proportions 0.1, 0.2, 0.3, 0.2, 0.2
            desired_proportions = {1: 0.2, 2: 0.1, 3: 0.2, 4: 0.2, 5: 0.3}

            # Calculate the number of samples needed for each category
            total_samples = 1000
            sample_sizes = {score: int(desired_proportions[score] * total_samples) for score in desired_proportions}

            # Create an empty DataFrame to store sampling results
            sampled_data = pd.DataFrame()

            # Sample each category
            for score, size in sample_sizes.items():
                # Get data for the current category
                class_data = df[df['output_field'] == score]
                # If the amount of data in the current category is less than the number of samples needed, add all data to the sampling result
                if len(class_data) <= size:
                    sampled_data = pd.concat([sampled_data, class_data])
                else:
                    sampled_data = pd.concat([sampled_data, class_data.sample(size, replace=False)])
            
            return sampled_data

        final_df = sample_by_kind(final_df)
        final_df = final_df.sample(frac=1)
        print(final_df['output_field'].value_counts())

        csv_path = json_file_path.replace("json", "csv")
        final_df[['prompt', 'output_field']].to_json(f"sentiment/{json_file_path}", orient="records", lines=True)
        final_df[['prompt', 'output_field']].to_csv(f"sentiment/{csv_path}", index=None)


Processing files:   0%|                                                                         | 0/32 [00:00<?, ?it/s]

All_Beauty_5.json *************************
5    300
4    152
3     64
1     30
2     26
Name: output_field, dtype: int64
AMAZON_FASHION_5.json *************************
5    291
4     66
3     49
1     17
2     15
Name: output_field, dtype: int64
Appliances_5.json *************************
5    113
4     12
2      7
3      7
1      5
Name: output_field, dtype: int64
Arts_Crafts_and_Sewing_5.json *************************


Processing files:  16%|██████████▏                                                      | 5/32 [00:00<00:03,  6.93it/s]

5    300
1    200
3    200
4    200
2    100
Name: output_field, dtype: int64
Automotive_5.json *************************


Processing files:  19%|████████████▏                                                    | 6/32 [00:01<00:06,  3.96it/s]

...

5    300
1    200
3    200
4    200
2    100
Name: output_field, dtype: int64
Toys_and_Games_5.json *************************


Processing files:  94%|████████████████████████████████████████████████████████████    | 30/32 [00:16<00:01,  1.68it/s]

5    300
1    200
3    200
4    200
2    100
Name: output_field, dtype: int64
Video_Games_5.json *************************


Processing files: 100%|████████████████████████████████████████████████████████████████| 32/32 [00:17<00:00,  1.82it/s]

5    300
1    200
3    200
4    200
2    100
Name: output_field, dtype: int64


In [6]:
all_part_data = [pd.read_csv(f"sentiment/{i}") for i in os.listdir("sentiment") if i.endswith(".csv")]
all_part_data = pd.concat(all_part_data)
all_part_data

,prompt,output_field
0,Instructions: Evaluate the following product r...,5
1,Instructions: Evaluate the following product r...,5
2,Instructions: Evaluate the following product r...,1
3,Instructions: Evaluate the following product r...,5
4,Instructions: Evaluate the following product r...,3
...,...,...
995,Instructions: Evaluate the following product r...,1
996,Instructions: Evaluate the following product r...,1
997,Instructions: Evaluate the following product r...,5
998,Instructions: Evaluate the following product r...,4
